In [3]:
import sympy as sp
import pandas as pd
import os
import matplotlib.pyplot as plt

**P(S|T) = Chance you have the SNP if you have the trait**

**P(T|S) = Chance you have the trait if you have the SNP**

**P(S) = Frequency of SNP in the population**

**P(T) = Frequency of the trait in the population**


***P(S|T) is what we are looking for***

P(S|T) = What is the probability someone has a certain SNP if we know they have the trait?

**Bayes Theorem**

$P(S|T) = \frac{P(T|S)P(S)}{P(T)}$

**What We Have**

OR = Odds Ratio: how strongly SNP is linked to trait  
P(S) = Risk Allele Frequency (RAF)  
P(T) = How common the trait is in the population  

In [4]:
w, OR, PS, PT = sp.symbols('w OR PS PT', real=True, positive=True)

**Step 1**
Estimate P(T|S) from OR

OR = (Odds of trait with SNP)/(Odds of trait without SNP)
# = $\frac{\frac{P(T|S)}{P(\overline{T}|S)}}{\frac{P(T|\overline{S})}{P(\overline{T}|\overline{S})}}$ #

Using conditional probability formula:

$P(T|S) = \frac{P({T}\cap{S})}{P(S)}$  
$P(\overline{T}|S) = \frac{P(\overline{T}\cap{S})}{P(S)}$  
$P(T|\overline{S}) = \frac{P({T}\cap\overline{S})}{P(\overline{S})}$  
$P(\overline{T}|\overline{S}) = \frac{P(\overline{T}\cap\overline{S})}{P(\overline{S})}$  

Plugging them into OR gives:

$\frac{P({T}\cap{S})P(\overline{T}\cap\overline{S})}{P(\overline{T}\cap{S})P(T\cap\overline{S})}$

Let w,x,y,z be following:

$w = P({T}\cap{S})$   
$x = P(\overline{T}\cap{S})$  
$y = P(T\cap\overline{S})$  
$z = P(\overline{T}\cap\overline{S})$  

Then, $OR = \frac{wz}{xy}$

Also,   
w + x = P(S)  
y + z = 1 - P(S)  
w + y = P(T)  
x + z = 1 - P(T)  

Thus,  
x = P(S) - w  
y = P(T) - w  
z = 1 - P(S) - P(T) + w

Plugging them into OR gives:
$OR = \frac{wz}{xy}$
   $= \frac{w(1-P(S)-P(T)+w)}{(P(S)-w)(P(T)-w)}$

Solve it for w using the formula.

In [5]:
def get_p_trait_cap_snp(OR_val, PS_val, PT_val):
    # define symbols
    w = sp.Symbol('w', real = True)
    PS = sp.Symbol('PS', real = True)
    PT = sp.Symbol('PT', real = True)
    OR = sp.Symbol('OR', real = True)

    # define equation
    expr = sp.Eq(OR, (w * (1 - PS - PT + w)) / ((PS - w) * (PT - w)))

    # solve for w
    solutions = sp.solve(expr, w)

    # substitute numbers
    numeric_solutions = [sol.evalf(subs={OR: OR_val, PS: PS_val, PT: PT_val}) for sol in solutions]

    valid_sol = []
    # filter solutions
    for sol in numeric_solutions:
        if sol.is_real and 0<= sol <= min(PS_val, PT_val):
            valid_sol.append(round(sol,6))
    return valid_sol

Now we have $P(T \cap S)$.

Then we calculate x,y,z using w, P(S), and P(T)

x ( = $P(\overline{T}\cap S)$)

In [6]:
def get_p_non_trait_cap_snp(p_trait_cap_snp, PS_val):
    return PS_val - p_trait_cap_snp

y ( = $P(T\cap\overline{S})$)

In [7]:
def get_p_trait_cap_non_snp(p_trait_cap_snp, PT_val):
    return PT_val - p_trait_cap_snp

z ( = $P(\overline{T}\cap\overline{S})$)

In [8]:
def get_p_non_trait_cap_non_snp(p_trait_cap_snp, PS_val, PT_val):
    return 1 - PS_val - PT_val + p_trait_cap_snp

$P(S|T) = \frac{P(T\cap S)}{P(T)}$

In [9]:
def get_p_snp_given_trait(p_trait_cap_snp, PT):
    if PT == 0:
        return None  # Avoid division by zero
    result = p_trait_cap_snp / PT
    return round(result, 6)


$P(S|T)$ with confidence interval

In [10]:
def p_snp_given_trait_from_OR_with_CI(OR_low, OR_mid, OR_high, PS_val, PT_val):
    result = {}
    for label, OR_val in zip(["lower", "point", "upper"], [OR_low, OR_mid, OR_high]):
        p_trait_cap_snp = get_p_trait_cap_snp(OR_val, PS_val, PT_val)
        if p_trait_cap_snp:
            p_snp_given_trait = get_p_snp_given_trait(p_trait_cap_snp[0], PT_val)
            result[label] = round(p_snp_given_trait, 6)
        else:
            result[label] = None
    return result

\begin{align}
    pRR &= \frac{P(S \mid T)}{P(S \mid\overline{T})} \\
    &= \frac{w}{P(T)}/\frac{P(S) - w}{1 - P(T)} \\
    &= \frac{w[1-P(T)]}{P(T)[P(S)-w]}
\end{align}

In [17]:
def get_pRR(p_trait_cap_snp, PS_val, PT_val):
    numerator = float(p_trait_cap_snp) * (1 - PT_val)
    denominator = PT_val * (PS_val - p_trait_cap_snp)
    return numerator / denominator

Get pRR with confidence interval

In [18]:
def get_pRR_from_OR_with_CI(OR_low, OR_mid, OR_high, PS_val, PT_val):
    result = {}
    for label, OR_val in zip(["lower", "point", "upper"], [OR_low, OR_mid, OR_high]):
        p_trait_cap_snp = get_p_trait_cap_snp(OR_val, PS_val, PT_val)
        if p_trait_cap_snp:
            prr = get_pRR(p_trait_cap_snp[0], PS_val, PT_val)
            result[label] = round(prr, 6)
        else:
            result[label] = None
    return result




Example:

APOE e4/e4 genotype  
trait: Alzheimer's disease 

OR: 13.52 CI[10.64 - 17.18]  
P(S): 0.024  
P(T): 0.001699729640318956





In [19]:
OR_low = 10.64
OR_mid = 13.52
OR_high = 17.18
PS_val = 0.024
PT_val = 0.001699729640318956

pRRs = get_pRR_from_OR_with_CI(OR_low, OR_mid, OR_high, PS_val, PT_val)

In [20]:
pRRs

{'lower': 8.666397, 'point': 10.436501, 'upper': 12.446754}